# AI-Powered Malaysian Legal PDF Analysis
## Extract PDF Content with Unstructured + Label Context with OpenAI/Gemini

This notebook demonstrates how to:
1. Extract text from Malaysian legal PDFs using the `unstructured` library
2. Use AI models (OpenAI GPT-4 or Google Gemini) to intelligently label and categorize content
3. Optimize for Google Colab GPU environment
4. Export structured, labeled results for legal AI/RAG systems

**Perfect for:** Legal document processing, content classification, and building intelligent legal knowledge bases

---

### 🚀 **Optimized for Google Colab GPU Runtime**
- Make sure to enable GPU: Runtime → Change runtime type → Hardware accelerator: GPU

## 📦 Install Required Libraries

Install all necessary packages for PDF processing and AI model integration.

In [ ]:
# Install required packages for Google Colab
!pip install -q unstructured[pdf] 
!pip install -q openai>=1.0.0
!pip install -q google-generativeai
!pip install -q python-dotenv
!pip install -q tqdm
!pip install -q pandas
!pip install -q matplotlib
!pip install -q seaborn

# Install additional dependencies for unstructured PDF processing
!apt-get update -qq
!apt-get install -qq poppler-utils tesseract-ocr

print("✅ All packages installed successfully!")

## 🔧 Import Dependencies and Setup API Keys

Import all necessary libraries and configure secure API key handling.

In [ ]:
import os
import json
import time
from pathlib import Path
from typing import List, Dict, Any, Optional
import logging
from datetime import datetime

# PDF Processing
from unstructured.partition.pdf import partition_pdf

# AI Model Libraries
import openai
import google.generativeai as genai

# Data Processing
import pandas as pd
from tqdm import tqdm

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Setup logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

print("✅ All libraries imported successfully!")

# API Key Setup
print("\n🔑 API Key Configuration:")
print("Please set your API keys using one of these methods:")
print("1. Use Colab Secrets (recommended)")
print("2. Set environment variables")
print("3. Direct assignment (not recommended for production)")

# Secure API key handling for Colab
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
    print("✅ Using Colab secrets for API keys")
except:
    # Fallback to environment variables or manual input
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
    
    if not OPENAI_API_KEY and not GEMINI_API_KEY:
        print("⚠️  No API keys found. Please set them manually:")
        print("OPENAI_API_KEY = 'your-openai-key-here'")
        print("GEMINI_API_KEY = 'your-gemini-key-here'")

# Set API keys
if OPENAI_API_KEY:
    openai.api_key = OPENAI_API_KEY
    print("✅ OpenAI API key configured")

if GEMINI_API_KEY:
    genai.configure(api_key=GEMINI_API_KEY)
    print("✅ Gemini API key configured")

## 🚀 Configure GPU and Environment

Check GPU availability and optimize environment for efficient processing.

In [ ]:
# Check GPU availability
try:
    import torch
    gpu_available = torch.cuda.is_available()
    if gpu_available:
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1024**3
        print(f"✅ GPU Available: {gpu_name}")
        print(f"📊 GPU Memory: {gpu_memory:.1f} GB")
        
        # Set optimal settings for GPU
        torch.backends.cudnn.benchmark = True
    else:
        print("⚠️  No GPU detected. Using CPU (will be slower)")
except ImportError:
    print("⚠️  PyTorch not available. Install if you need GPU acceleration.")

# Check system resources
import psutil
cpu_count = psutil.cpu_count()
memory_gb = psutil.virtual_memory().total / 1024**3

print(f"🖥️  CPU cores: {cpu_count}")
print(f"💾 RAM: {memory_gb:.1f} GB")

# Optimize environment settings
os.environ['TOKENIZERS_PARALLELISM'] = 'false'  # Avoid tokenizer warnings
os.environ['OMP_NUM_THREADS'] = str(min(4, cpu_count))  # Optimize CPU usage

print("✅ Environment optimized for processing")

## 📄 Load and Extract PDF Content

Use unstructured library to extract elements from Malaysian legal PDFs with high accuracy.

In [ ]:
def extract_pdf_content(pdf_path: str, max_pages: Optional[int] = None) -> List[Dict[str, Any]]:
    """
    Extract content from PDF using unstructured library with high accuracy settings.
    
    Args:
        pdf_path: Path to the PDF file
        max_pages: Maximum number of pages to process (None for all pages)
    
    Returns:
        List of extracted elements with metadata
    """
    logger.info(f"🔍 Extracting content from: {Path(pdf_path).name}")
    
    try:
        # Configure extraction settings for maximum accuracy
        elements = partition_pdf(
            filename=pdf_path,
            strategy="hi_res",  # High resolution for legal documents
            infer_table_structure=True,  # Detect tables
            extract_images_in_pdf=False,  # Skip images for text focus
            include_page_breaks=True,  # Preserve page structure
        )
        
        logger.info(f"✅ Extracted {len(elements)} elements from entire PDF")
        
        # Convert to structured format
        structured_elements = []
        
        for i, element in enumerate(elements):
            # Get element metadata
            page_num = 1  # default
            coordinates = None
            
            if hasattr(element, 'metadata') and element.metadata:
                if hasattr(element.metadata, 'page_number'):
                    page_num = element.metadata.page_number
                if hasattr(element.metadata, 'coordinates'):
                    coordinates = str(element.metadata.coordinates)
            
            # Filter by page limit if specified
            if max_pages and page_num > max_pages:
                continue
            
            # Create structured element
            structured_element = {
                'index': i,
                'page_number': page_num,
                'element_type': str(type(element).__name__),
                'text': str(element).strip(),
                'character_count': len(str(element)),
                'coordinates': coordinates,
                'metadata': {
                    'extraction_timestamp': datetime.now().isoformat(),
                    'source_file': Path(pdf_path).name
                }
            }
            
            structured_elements.append(structured_element)
        
        # Filter and sort
        filtered_elements = [e for e in structured_elements if e['character_count'] > 5]  # Remove tiny elements
        filtered_elements.sort(key=lambda x: (x['page_number'], x['index']))  # Sort by page and order
        
        logger.info(f"📊 Final elements: {len(filtered_elements)} (after filtering)")
        return filtered_elements
        
    except Exception as e:
        logger.error(f"❌ Error extracting PDF: {e}")
        return []

# Example usage - Upload your PDF to Colab or use a sample
print("📁 To use this function:")
print("1. Upload your Malaysian legal PDF to Colab")
print("2. Update the pdf_path variable below")
print("3. Run the extraction")

# Sample extraction (update this path)
pdf_path = "/content/sample_legal_document.pdf"  # Update this path
max_pages = 3  # Process only first 3 pages for testing

print(f"\\n🔄 Ready to extract from: {pdf_path}")
print(f"📄 Max pages: {max_pages}")

# Uncomment the next line when you have a PDF ready
# elements = extract_pdf_content(pdf_path, max_pages=max_pages)

## 🤖 Setup AI Model Clients

Initialize OpenAI GPT and Google Gemini clients for intelligent content labeling.

In [ ]:
class AILabelingSystem:
    """Unified system for labeling PDF content using OpenAI or Gemini"""
    
    def __init__(self):
        self.openai_client = None
        self.gemini_model = None
        self.setup_clients()
    
    def setup_clients(self):
        """Initialize available AI model clients"""
        
        # Setup OpenAI client
        if OPENAI_API_KEY:
            try:
                self.openai_client = openai.OpenAI(api_key=OPENAI_API_KEY)
                logger.info("✅ OpenAI client initialized")
            except Exception as e:
                logger.error(f"❌ OpenAI setup failed: {e}")
        
        # Setup Gemini client
        if GEMINI_API_KEY:
            try:
                self.gemini_model = genai.GenerativeModel('gemini-1.5-flash')
                logger.info("✅ Gemini client initialized")
            except Exception as e:
                logger.error(f"❌ Gemini setup failed: {e}")
        
        if not self.openai_client and not self.gemini_model:
            logger.warning("⚠️  No AI models available. Please configure API keys.")
    
    def test_models(self):
        """Test both models with a simple query"""
        test_text = "This is a test of the legal document analysis system."
        
        print("🧪 Testing AI Models:\\n")
        
        # Test OpenAI
        if self.openai_client:
            try:
                response = self.openai_client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=[
                        {"role": "user", "content": f"Classify this text: {test_text}"}
                    ],
                    max_tokens=50
                )
                print("✅ OpenAI GPT: Working")
                print(f"   Response: {response.choices[0].message.content.strip()}")
            except Exception as e:
                print(f"❌ OpenAI test failed: {e}")
        
        # Test Gemini
        if self.gemini_model:
            try:
                response = self.gemini_model.generate_content(
                    f"Classify this text in one sentence: {test_text}"
                )
                print("✅ Gemini: Working")
                print(f"   Response: {response.text.strip()}")
            except Exception as e:
                print(f"❌ Gemini test failed: {e}")

# Initialize the AI labeling system
ai_system = AILabelingSystem()

# Test the models
ai_system.test_models()

## 🏷️ Create Context Labeling Functions

Define intelligent functions to classify and label legal document content using AI models.

In [ ]:
def create_labeling_prompt(text: str, element_type: str) -> str:
    """Create a sophisticated prompt for legal content classification"""
    
    prompt = f"""
You are an expert legal document analyst specializing in Malaysian law. 
Analyze the following text from a legal document and provide a comprehensive classification.

TEXT TO ANALYZE:
"{text}"

ELEMENT TYPE: {element_type}

Please provide a JSON response with the following fields:
1. "content_category": Main category (e.g., "section_header", "legal_definition", "procedural_requirement", "penalty_clause", "table_of_contents", "preamble", "schedule", "interpretation")
2. "legal_significance": Level of legal importance (e.g., "high", "medium", "low")
3. "subject_matter": What the text is about (e.g., "criminal_procedure", "corporate_law", "tax_provisions", "regulatory_compliance")
4. "contains_definitions": Whether it contains legal definitions (true/false)
5. "contains_penalties": Whether it mentions penalties or sanctions (true/false)
6. "references_other_sections": Whether it references other legal sections (true/false)
7. "actionable_requirements": Whether it contains specific requirements or obligations (true/false)
8. "confidence_score": Your confidence in this classification (0.0 to 1.0)
9. "keywords": Key legal terms found in the text (array)
10. "summary": Brief summary of the content (1-2 sentences)

Respond with valid JSON only.
"""
    return prompt.strip()

def label_with_openai(ai_system, text: str, element_type: str) -> Dict[str, Any]:
    """Label content using OpenAI GPT"""
    try:
        prompt = create_labeling_prompt(text, element_type)
        
        response = ai_system.openai_client.chat.completions.create(
            model="gpt-4",  # Use GPT-4 for better accuracy
            messages=[
                {"role": "system", "content": "You are a legal document analysis expert. Always respond with valid JSON."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.1  # Low temperature for consistent results
        )
        
        # Parse JSON response
        content = response.choices[0].message.content.strip()
        if content.startswith('```json'):
            content = content[7:]
        if content.endswith('```'):
            content = content[:-3]
        
        result = json.loads(content)
        result['model_used'] = 'gpt-4'
        result['processing_time'] = response.usage.total_tokens if hasattr(response, 'usage') else None
        
        return result
        
    except Exception as e:
        logger.error(f"OpenAI labeling failed: {e}")
        return {
            "content_category": "unknown",
            "error": str(e),
            "model_used": "gpt-4"
        }

def label_with_gemini(ai_system, text: str, element_type: str) -> Dict[str, Any]:
    """Label content using Google Gemini"""
    try:
        prompt = create_labeling_prompt(text, element_type)
        
        response = ai_system.gemini_model.generate_content(
            prompt,
            generation_config=genai.types.GenerationConfig(
                temperature=0.1,
                max_output_tokens=500,
            )
        )
        
        # Parse JSON response
        content = response.text.strip()
        if content.startswith('```json'):
            content = content[7:]
        if content.endswith('```'):
            content = content[:-3]
        
        result = json.loads(content)
        result['model_used'] = 'gemini-1.5-flash'
        
        return result
        
    except Exception as e:
        logger.error(f"Gemini labeling failed: {e}")
        return {
            "content_category": "unknown", 
            "error": str(e),
            "model_used": "gemini-1.5-flash"
        }

def smart_label_content(ai_system, text: str, element_type: str, preferred_model: str = "auto") -> Dict[str, Any]:
    """
    Intelligently label content using the best available model
    
    Args:
        ai_system: AILabelingSystem instance
        text: Text content to label
        element_type: Type of element from unstructured
        preferred_model: "openai", "gemini", or "auto"
    """
    
    # Skip very short text
    if len(text.strip()) < 10:
        return {
            "content_category": "minimal_content",
            "confidence_score": 0.9,
            "model_used": "rule_based"
        }
    
    # Choose model
    if preferred_model == "auto":
        # Prefer OpenAI for complex analysis, Gemini for speed
        if ai_system.openai_client and len(text) > 200:
            preferred_model = "openai"
        elif ai_system.gemini_model:
            preferred_model = "gemini"
        elif ai_system.openai_client:
            preferred_model = "openai"
    
    # Label with chosen model
    if preferred_model == "openai" and ai_system.openai_client:
        return label_with_openai(ai_system, text, element_type)
    elif preferred_model == "gemini" and ai_system.gemini_model:
        return label_with_gemini(ai_system, text, element_type)
    else:
        logger.warning("No AI model available for labeling")
        return {
            "content_category": "unlabeled",
            "error": "No AI model available",
            "model_used": "none"
        }

print("✅ Context labeling functions created!")
print("🎯 Available functions:")
print("   • smart_label_content() - Auto-select best model")
print("   • label_with_openai() - Use GPT-4")
print("   • label_with_gemini() - Use Gemini")
print("\\n🧪 Test with a sample:")

# Test the labeling system
sample_text = "31A. Establishment of the Commission\\n(1) There is established a commission to be known as the Malaysian Communications and Multimedia Commission."
sample_result = smart_label_content(ai_system, sample_text, "NarrativeText")
print(f"Sample result: {json.dumps(sample_result, indent=2)}")

## 🔄 Process PDF Elements with AI

Extract PDF content and apply intelligent AI labeling to each element.

In [ ]:
def process_pdf_with_ai_labeling(
    pdf_path: str, 
    max_pages: Optional[int] = None,
    preferred_model: str = "auto",
    rate_limit_delay: float = 1.0,
    batch_size: int = 10
) -> Dict[str, Any]:
    """
    Complete pipeline: Extract PDF content and label with AI
    
    Args:
        pdf_path: Path to PDF file
        max_pages: Max pages to process
        preferred_model: "openai", "gemini", or "auto"
        rate_limit_delay: Delay between API calls (seconds)
        batch_size: Process elements in batches
    
    Returns:
        Complete analysis results
    """
    
    logger.info(f"🚀 Starting AI-powered PDF analysis: {Path(pdf_path).name}")
    start_time = time.time()
    
    # Step 1: Extract PDF content
    elements = extract_pdf_content(pdf_path, max_pages)
    if not elements:
        return {"error": "Failed to extract PDF content", "elements": []}
    
    logger.info(f"📄 Processing {len(elements)} elements with AI labeling...")
    
    # Step 2: Process elements with AI
    labeled_elements = []
    api_calls_made = 0
    errors = 0
    
    # Process in batches with progress bar
    for i in tqdm(range(0, len(elements), batch_size), desc="🤖 AI Labeling"):
        batch = elements[i:i + batch_size]
        
        for element in batch:
            try:
                # AI labeling
                ai_label = smart_label_content(
                    ai_system, 
                    element['text'], 
                    element['element_type'], 
                    preferred_model
                )
                
                # Combine original element with AI labels
                labeled_element = {
                    **element,  # Original element data
                    'ai_labels': ai_label,  # AI-generated labels
                    'processing_timestamp': datetime.now().isoformat()
                }
                
                labeled_elements.append(labeled_element)
                api_calls_made += 1
                
                # Rate limiting
                time.sleep(rate_limit_delay)
                
            except Exception as e:
                logger.error(f"Error processing element {element['index']}: {e}")
                errors += 1
                
                # Add element without AI labels
                labeled_elements.append({
                    **element,
                    'ai_labels': {'error': str(e), 'content_category': 'processing_failed'},
                    'processing_timestamp': datetime.now().isoformat()
                })
    
    # Step 3: Generate analysis summary
    processing_time = time.time() - start_time
    
    # Analyze results
    categories = {}
    high_importance = 0
    total_chars = 0
    
    for element in labeled_elements:
        ai_labels = element.get('ai_labels', {})
        category = ai_labels.get('content_category', 'unknown')
        categories[category] = categories.get(category, 0) + 1
        
        if ai_labels.get('legal_significance') == 'high':
            high_importance += 1
        
        total_chars += element.get('character_count', 0)
    
    # Compile final results
    results = {
        'document_info': {
            'source_file': Path(pdf_path).name,
            'total_elements': len(labeled_elements),
            'total_characters': total_chars,
            'pages_processed': max_pages if max_pages else "all",
            'processing_time_seconds': round(processing_time, 2),
            'api_calls_made': api_calls_made,
            'errors': errors
        },
        'analysis_summary': {
            'content_categories': categories,
            'high_importance_elements': high_importance,
            'category_distribution': {k: round(v/len(labeled_elements)*100, 1) for k, v in categories.items()},
            'avg_chars_per_element': round(total_chars / len(labeled_elements)) if labeled_elements else 0
        },
        'labeled_elements': labeled_elements
    }
    
    logger.info(f"✅ Analysis complete! Processed {len(labeled_elements)} elements in {processing_time:.1f}s")
    return results

# Example processing function
def run_example_analysis(pdf_path: str = "/content/sample_legal_document.pdf"):
    """Run a complete example analysis"""
    
    if not Path(pdf_path).exists():
        print(f"❌ PDF not found: {pdf_path}")
        print("📁 Please upload your PDF to Colab and update the path")
        return None
    
    print(f"🔍 Analyzing: {pdf_path}")
    print("⚙️  Settings: First 3 pages, auto model selection, 1s rate limit")
    
    # Run analysis
    results = process_pdf_with_ai_labeling(
        pdf_path=pdf_path,
        max_pages=3,  # First 3 pages only
        preferred_model="auto",
        rate_limit_delay=1.0,
        batch_size=5
    )
    
    if results and 'document_info' in results:
        # Display summary
        info = results['document_info']
        summary = results['analysis_summary']
        
        print(f"\\n📊 ANALYSIS RESULTS:")
        print(f"📄 Elements processed: {info['total_elements']}")
        print(f"⏱️  Processing time: {info['processing_time_seconds']}s")
        print(f"🤖 API calls made: {info['api_calls_made']}")
        print(f"⚠️  Errors: {info['errors']}")
        
        print(f"\\n🏷️  CONTENT CATEGORIES:")
        for category, count in summary['content_categories'].items():
            percentage = summary['category_distribution'][category]
            print(f"   • {category}: {count} ({percentage}%)")
        
        print(f"\\n🎯 High importance elements: {summary['high_importance_elements']}")
        
        return results
    
    return None

print("✅ PDF processing functions ready!")
print("\\n🚀 To run analysis:")
print("1. Upload your PDF to Colab")
print("2. Update the pdf_path in run_example_analysis()")
print("3. Call: results = run_example_analysis()")

# Uncomment to run with your PDF
# results = run_example_analysis("/content/your_legal_document.pdf")

## ⚡ Batch Processing and Optimization

Advanced techniques for processing multiple PDFs efficiently with caching and GPU optimization.

In [ ]:
import hashlib
import pickle
from concurrent.futures import ThreadPoolExecutor, as_completed

class OptimizedPDFProcessor:
    """Advanced PDF processor with caching and batch optimization"""
    
    def __init__(self, cache_dir: str = "/content/pdf_cache"):
        self.cache_dir = Path(cache_dir)
        self.cache_dir.mkdir(exist_ok=True)
        self.processed_files = {}
        
    def get_file_hash(self, file_path: str) -> str:
        """Generate hash for file caching"""
        with open(file_path, 'rb') as f:
            return hashlib.md5(f.read()).hexdigest()
    
    def load_from_cache(self, file_hash: str) -> Optional[Dict]:
        """Load processed results from cache"""
        cache_file = self.cache_dir / f"{file_hash}.pkl"
        if cache_file.exists():
            try:
                with open(cache_file, 'rb') as f:
                    return pickle.load(f)
            except Exception as e:
                logger.warning(f"Cache load failed: {e}")
        return None
    
    def save_to_cache(self, file_hash: str, results: Dict):
        """Save results to cache"""
        cache_file = self.cache_dir / f"{file_hash}.pkl"
        try:
            with open(cache_file, 'wb') as f:
                pickle.dump(results, f)
        except Exception as e:
            logger.warning(f"Cache save failed: {e}")
    
    def process_single_pdf_optimized(
        self, 
        pdf_path: str,
        use_cache: bool = True,
        **kwargs
    ) -> Dict:
        """Process single PDF with caching"""
        
        file_hash = self.get_file_hash(pdf_path)
        
        # Check cache first
        if use_cache:
            cached_result = self.load_from_cache(file_hash)
            if cached_result:
                logger.info(f"📁 Loaded from cache: {Path(pdf_path).name}")
                return cached_result
        
        # Process if not cached
        logger.info(f"🔄 Processing (not cached): {Path(pdf_path).name}")
        results = process_pdf_with_ai_labeling(pdf_path, **kwargs)
        
        # Save to cache
        if use_cache and results:
            self.save_to_cache(file_hash, results)
        
        return results
    
    def batch_process_pdfs(
        self,
        pdf_paths: List[str],
        max_workers: int = 3,  # Conservative for API rate limits
        use_cache: bool = True,
        **kwargs
    ) -> Dict[str, Any]:
        """Process multiple PDFs in parallel with optimization"""
        
        logger.info(f"🚀 Starting batch processing of {len(pdf_paths)} PDFs")
        
        results = {}
        total_start_time = time.time()
        
        # Process with controlled parallelism
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all jobs
            future_to_path = {
                executor.submit(
                    self.process_single_pdf_optimized, 
                    pdf_path, 
                    use_cache, 
                    **kwargs
                ): pdf_path for pdf_path in pdf_paths
            }
            
            # Collect results with progress bar
            for future in tqdm(as_completed(future_to_path), total=len(pdf_paths), desc="📄 Processing PDFs"):
                pdf_path = future_to_path[future]
                try:
                    result = future.result()
                    results[Path(pdf_path).name] = result
                except Exception as e:
                    logger.error(f"Error processing {pdf_path}: {e}")
                    results[Path(pdf_path).name] = {"error": str(e)}
        
        # Generate batch summary
        total_time = time.time() - total_start_time
        successful = sum(1 for r in results.values() if "error" not in r)
        failed = len(results) - successful
        
        batch_summary = {
            'batch_info': {
                'total_files': len(pdf_paths),
                'successful': successful,
                'failed': failed,
                'total_processing_time': round(total_time, 2),
                'avg_time_per_file': round(total_time / len(pdf_paths), 2)
            },
            'file_results': results
        }
        
        logger.info(f"✅ Batch complete: {successful}/{len(pdf_paths)} successful in {total_time:.1f}s")
        return batch_summary

def demonstrate_optimization():
    """Demonstrate advanced processing features"""
    
    print("⚡ OPTIMIZATION FEATURES:")
    print("\\n1. 📁 Intelligent Caching:")
    print("   • Files are hashed to detect changes")
    print("   • Processed results cached to disk")
    print("   • Automatic cache invalidation")
    
    print("\\n2. 🔄 Parallel Processing:")
    print("   • Multiple PDFs processed simultaneously")
    print("   • Controlled concurrency for API limits")
    print("   • Progress tracking and error handling")
    
    print("\\n3. 🎯 Memory Optimization:")
    print("   • Batch processing to manage memory")
    print("   • Garbage collection between files")
    print("   • GPU memory monitoring")
    
    print("\\n4. 📊 Performance Monitoring:")
    print("   • Processing time tracking")
    print("   • API call counting")
    print("   • Success/failure rates")
    
    # Example usage
    processor = OptimizedPDFProcessor()
    
    print(f"\\n💾 Cache directory: {processor.cache_dir}")
    print(f"📁 Cache size: {len(list(processor.cache_dir.glob('*.pkl')))} files")
    
    return processor

# Initialize optimized processor
processor = demonstrate_optimization()

# Example batch processing
def run_batch_example(pdf_directory: str = "/content/legal_pdfs/"):
    """Example of batch processing multiple PDFs"""
    
    pdf_dir = Path(pdf_directory)
    if not pdf_dir.exists():
        print(f"❌ Directory not found: {pdf_directory}")
        print("📁 Create directory and upload PDFs to test batch processing")
        return
    
    # Find all PDFs
    pdf_files = list(pdf_dir.glob("*.pdf"))
    if not pdf_files:
        print(f"❌ No PDF files found in {pdf_directory}")
        return
    
    print(f"🔍 Found {len(pdf_files)} PDF files")
    
    # Run batch processing
    batch_results = processor.batch_process_pdfs(
        pdf_paths=[str(p) for p in pdf_files],
        max_workers=2,  # Conservative for API limits
        max_pages=2,    # First 2 pages only for demo
        preferred_model="auto",
        rate_limit_delay=0.5
    )
    
    # Display results
    print("\\n📊 BATCH RESULTS:")
    info = batch_results['batch_info']
    print(f"✅ Successful: {info['successful']}/{info['total_files']}")
    print(f"⏱️  Total time: {info['total_processing_time']}s")
    print(f"📈 Avg per file: {info['avg_time_per_file']}s")
    
    return batch_results

print("\\n🚀 Ready for optimized processing!")
print("💡 Tips for Google Colab:")
print("   • Use GPU runtime for faster processing")
print("   • Enable high-RAM if processing many files")
print("   • Monitor API usage to avoid rate limits")
print("   • Cache results to avoid reprocessing")

## 📊 Export Results and Visualization

Save labeled PDF content and create insightful visualizations of the analysis results.

In [ ]:
def export_results_to_formats(results: Dict, output_dir: str = "/content/parsed/EN"):
    """Export Malaysian legal analysis results to multiple formats in parsed folder structure"""
    
    output_path = Path(output_dir)
    output_path.mkdir(parents=True, exist_ok=True)
    
    if 'document_info' not in results:
        logger.error("Invalid results format")
        return
    
    doc_info = results['document_info']
    doc_name = doc_info['source_file'].replace('.pdf', '')
    language = doc_info.get('language', 'unknown')
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # 1. Complete JSON export (this is the main output for parsed folder)
    json_file = output_path / f"{doc_name}_complete_analysis.json"
    with open(json_file, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=2, ensure_ascii=False)
    
    # 2. Structured CSV for legal review and analysis
    elements_data = []
    for element in results['labeled_elements']:
        ai_labels = element.get('ai_labels', {})
        row = {
            'document_name': doc_name,
            'language': language,
            'index': element['index'],
            'page_number': element['page_number'],
            'element_type': element['element_type'],
            'character_count': element['character_count'],
            'content_category': ai_labels.get('content_category', 'unknown'),
            'legal_significance': ai_labels.get('legal_significance', 'unknown'),
            'subject_matter': ai_labels.get('subject_matter', 'unknown'),
            'confidence_score': ai_labels.get('confidence_score', 0),
            'contains_definitions': ai_labels.get('contains_definitions', False),
            'contains_penalties': ai_labels.get('contains_penalties', False),
            'actionable_requirements': ai_labels.get('actionable_requirements', False),
            'references_other_sections': ai_labels.get('references_other_sections', False),
            'model_used': ai_labels.get('model_used', 'unknown'),
            'keywords': ', '.join(ai_labels.get('keywords', [])) if ai_labels.get('keywords') else '',
            'summary': ai_labels.get('summary', ''),
            'text_preview': element['text'][:200] + "..." if len(element['text']) > 200 else element['text']
        }
        elements_data.append(row)
    
    df = pd.DataFrame(elements_data)
    csv_file = output_path / f"{doc_name}_legal_analysis.csv"
    df.to_csv(csv_file, index=False, encoding='utf-8')
    
    # 3. Legal summary report for human review
    report_file = output_path / f"{doc_name}_legal_summary.txt"
    with open(report_file, 'w', encoding='utf-8') as f:
        info = results['document_info']
        summary = results['analysis_summary']
        
        f.write(f"MALAYSIAN LEGAL DOCUMENT ANALYSIS REPORT\\n")
        f.write(f"{'='*60}\\n\\n")
        f.write(f"Document: {info['source_file']}\\n")
        f.write(f"Language: {info.get('language', 'Unknown')}\\n")
        f.write(f"Source Path: {info.get('source_path', 'Unknown')}\\n")
        f.write(f"Processed: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\\n")
        f.write(f"Processing Time: {info['processing_time_seconds']}s\\n")
        f.write(f"Extraction Method: {info.get('extraction_method', 'Unknown')}\\n\\n")
        
        f.write(f"DOCUMENT STATISTICS:\\n")
        f.write(f"{'-'*30}\\n")
        f.write(f"Total Elements: {info['total_elements']}\\n")
        f.write(f"Total Characters: {info['total_characters']:,}\\n")
        f.write(f"API Calls Made: {info['api_calls_made']}\\n")
        f.write(f"Models Used: {', '.join(info.get('models_used', []))}\\n")
        f.write(f"Average Confidence: {summary['average_confidence']}\\n\\n")
        
        f.write(f"LEGAL CONTENT ANALYSIS:\\n")
        f.write(f"{'-'*30}\\n")
        for category, count in summary['content_categories'].items():
            percentage = summary['category_distribution'][category]
            f.write(f"{category}: {count} elements ({percentage}%)\\n")
        
        f.write(f"\\nLEGAL SIGNIFICANCE:\\n")
        f.write(f"{'-'*30}\\n")
        f.write(f"High Importance Elements: {summary['high_importance_elements']}\\n")
        
        # Extract high importance elements
        high_importance_elements = [
            el for el in results['labeled_elements'] 
            if el.get('ai_labels', {}).get('legal_significance') == 'high'
        ]
        
        if high_importance_elements:
            f.write(f"\\nHIGH IMPORTANCE SECTIONS:\\n")
            f.write(f"{'-'*30}\\n")
            for i, element in enumerate(high_importance_elements[:10]):  # Top 10
                ai_labels = element.get('ai_labels', {})
                f.write(f"{i+1}. Page {element['page_number']} - {ai_labels.get('content_category', 'Unknown')}\\n")
                f.write(f"   Subject: {ai_labels.get('subject_matter', 'Unknown')}\\n")
                f.write(f"   Text: {element['text'][:150]}{'...' if len(element['text']) > 150 else ''}\\n\\n")
    
    # 4. Create a metadata index file for the parsed folder
    metadata_file = output_path / f"{doc_name}_metadata.json"
    metadata = {
        'document_info': doc_info,
        'analysis_summary': summary,
        'files_generated': {
            'complete_analysis': json_file.name,
            'csv_analysis': csv_file.name,
            'text_summary': report_file.name,
            'metadata': metadata_file.name
        },
        'generation_timestamp': datetime.now().isoformat()
    }
    
    with open(metadata_file, 'w', encoding='utf-8') as f:
        json.dump(metadata, f, indent=2, ensure_ascii=False)
    
    logger.info(f"✅ Malaysian legal analysis exported to: {output_path}")
    logger.info(f"📄 Files created:")
    logger.info(f"   • {json_file.name} (Complete analysis - main output)")
    logger.info(f"   • {csv_file.name} (Structured data for review)")
    logger.info(f"   • {report_file.name} (Human-readable summary)")
    logger.info(f"   • {metadata_file.name} (Document metadata)")
    
    return {
        'json_file': str(json_file),
        'csv_file': str(csv_file),
        'report_file': str(report_file),
        'metadata_file': str(metadata_file),
        'dataframe': df,
        'output_directory': str(output_path)
    }

def create_analysis_visualizations(results: Dict, output_dir: str = "/content/parsed/EN"):
    """Create comprehensive visualizations for Malaysian legal document analysis"""
    
    if 'labeled_elements' not in results:
        logger.error("No labeled elements found for visualization")
        return
    
    doc_info = results['document_info']
    doc_name = doc_info['source_file'].replace('.pdf', '')
    language = doc_info.get('language', 'unknown')
    
    # Prepare data
    elements_data = []
    for element in results['labeled_elements']:
        ai_labels = element.get('ai_labels', {})
        elements_data.append({
            'page': element['page_number'],
            'element_type': element['element_type'],
            'category': ai_labels.get('content_category', 'unknown'),
            'significance': ai_labels.get('legal_significance', 'unknown'),
            'confidence': ai_labels.get('confidence_score', 0),
            'char_count': element['character_count'],
            'has_definitions': ai_labels.get('contains_definitions', False),
            'has_penalties': ai_labels.get('contains_penalties', False),
            'has_requirements': ai_labels.get('actionable_requirements', False)
        })
    
    df = pd.DataFrame(elements_data)
    
    # Create enhanced visualizations for legal documents
    fig, axes = plt.subplots(3, 3, figsize=(20, 15))
    fig.suptitle(f'Malaysian Legal Document Analysis: {doc_name} ({language})', fontsize=16, fontweight='bold')
    
    # 1. Content Categories Distribution
    category_counts = df['category'].value_counts()
    colors = plt.cm.Set3(range(len(category_counts)))
    axes[0, 0].pie(category_counts.values, labels=category_counts.index, autopct='%1.1f%%', 
                  startangle=90, colors=colors)
    axes[0, 0].set_title('Legal Content Categories')
    
    # 2. Legal Significance Distribution
    significance_counts = df['significance'].value_counts()
    colors_sig = {'high': 'red', 'medium': 'orange', 'low': 'green', 'unknown': 'gray'}
    bar_colors = [colors_sig.get(sig, 'gray') for sig in significance_counts.index]
    bars = axes[0, 1].bar(significance_counts.index, significance_counts.values, color=bar_colors)
    axes[0, 1].set_title('Legal Significance Levels')
    axes[0, 1].set_ylabel('Count')
    
    # 3. Elements Distribution by Page
    page_counts = df['page'].value_counts().sort_index()
    axes[0, 2].plot(page_counts.index, page_counts.values, marker='o', linewidth=2, markersize=6, color='blue')
    axes[0, 2].set_title('Elements per Page')
    axes[0, 2].set_xlabel('Page Number')
    axes[0, 2].set_ylabel('Element Count')
    axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Element Types Distribution
    type_counts = df['element_type'].value_counts()
    axes[1, 0].barh(type_counts.index, type_counts.values, color='skyblue')
    axes[1, 0].set_title('Document Element Types')
    axes[1, 0].set_xlabel('Count')
    
    # 5. AI Confidence Score Distribution
    axes[1, 1].hist(df['confidence'], bins=20, color='lightgreen', alpha=0.7, edgecolor='black')
    axes[1, 1].set_title('AI Confidence Score Distribution')
    axes[1, 1].set_xlabel('Confidence Score')
    axes[1, 1].set_ylabel('Frequency')
    mean_conf = df['confidence'].mean()
    axes[1, 1].axvline(mean_conf, color='red', linestyle='--', label=f'Mean: {mean_conf:.2f}')
    axes[1, 1].legend()
    
    # 6. Character Count vs Confidence (colored by page)
    scatter = axes[1, 2].scatter(df['char_count'], df['confidence'], c=df['page'], 
                               cmap='viridis', alpha=0.6, s=30)
    axes[1, 2].set_title('Content Length vs AI Confidence')
    axes[1, 2].set_xlabel('Character Count')
    axes[1, 2].set_ylabel('Confidence Score')
    plt.colorbar(scatter, ax=axes[1, 2], label='Page Number')
    
    # 7. Legal Features Analysis
    features = ['has_definitions', 'has_penalties', 'has_requirements']
    feature_counts = [df[feature].sum() for feature in features]
    feature_labels = ['Contains Definitions', 'Contains Penalties', 'Has Requirements']
    axes[2, 0].bar(feature_labels, feature_counts, color=['purple', 'orange', 'green'])
    axes[2, 0].set_title('Legal Features Detection')
    axes[2, 0].set_ylabel('Count')
    plt.setp(axes[2, 0].xaxis.get_majorticklabels(), rotation=45, ha='right')
    
    # 8. High Importance Elements by Page
    high_importance = df[df['significance'] == 'high']
    if not high_importance.empty:
        high_by_page = high_importance['page'].value_counts().sort_index()
        axes[2, 1].bar(high_by_page.index, high_by_page.values, color='red', alpha=0.7)
        axes[2, 1].set_title('High Importance Elements by Page')
        axes[2, 1].set_xlabel('Page Number')
        axes[2, 1].set_ylabel('High Importance Count')
    else:
        axes[2, 1].text(0.5, 0.5, 'No High Importance\\nElements Found', 
                       ha='center', va='center', transform=axes[2, 1].transAxes)
        axes[2, 1].set_title('High Importance Elements by Page')
    
    # 9. Category vs Significance Heatmap
    if len(df) > 0:
        pivot_table = df.pivot_table(values='confidence', index='category', 
                                   columns='significance', aggfunc='count', fill_value=0)
        if not pivot_table.empty:
            im = axes[2, 2].imshow(pivot_table.values, cmap='YlOrRd', aspect='auto')
            axes[2, 2].set_xticks(range(len(pivot_table.columns)))
            axes[2, 2].set_yticks(range(len(pivot_table.index)))
            axes[2, 2].set_xticklabels(pivot_table.columns)
            axes[2, 2].set_yticklabels(pivot_table.index)
            axes[2, 2].set_title('Category vs Significance Heatmap')
            plt.colorbar(im, ax=axes[2, 2])
        else:
            axes[2, 2].text(0.5, 0.5, 'Insufficient Data\\nfor Heatmap', 
                           ha='center', va='center', transform=axes[2, 2].transAxes)
            axes[2, 2].set_title('Category vs Significance Heatmap')
    
    plt.tight_layout()
    
    # Save visualization
    output_path = Path(output_dir)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    viz_file = output_path / f"{doc_name}_legal_analysis_dashboard.png"
    plt.savefig(viz_file, dpi=300, bbox_inches='tight')
    plt.show()
    
    logger.info(f"📊 Legal analysis dashboard saved: {viz_file}")
    
    # Enhanced analysis summary
    print(f"\\n? MALAYSIAN LEGAL DOCUMENT ANALYSIS SUMMARY:")
    print(f"📄 Document: {doc_name} ({language})")
    print(f"📈 Total elements: {len(df)}")
    print(f"🎯 Average AI confidence: {df['confidence'].mean():.2f}")
    print(f"📝 Average chars per element: {df['char_count'].mean():.0f}")
    print(f"⭐ High confidence elements (>0.8): {len(df[df['confidence'] > 0.8])}")
    print(f"🚨 High importance elements: {len(df[df['significance'] == 'high'])}")
    
    print(f"\\n🏷️  TOP LEGAL CATEGORIES:")
    for cat, count in category_counts.head().items():
        percentage = (count / len(df)) * 100
        print(f"   • {cat}: {count} ({percentage:.1f}%)")
    
    print(f"\\n⚖️  LEGAL FEATURES DETECTED:")
    print(f"   • Documents with definitions: {df['has_definitions'].sum()}")
    print(f"   • Documents with penalties: {df['has_penalties'].sum()}")
    print(f"   • Documents with requirements: {df['has_requirements'].sum()}")
    
    return df

def demonstrate_malaysian_legal_export():
    """Demonstrate export workflow for Malaysian legal documents"""
    
    print("📊 MALAYSIAN LEGAL DOCUMENT EXPORT WORKFLOW:")
    print("\\n1. 📄 Structured Output for Parsed Folder:")
    print("   • Complete JSON analysis (main output)")
    print("   • CSV file for legal review and analysis")
    print("   • Human-readable summary report")
    print("   • Metadata index for document tracking")
    
    print("\\n2. 📈 Legal-Specific Visualizations:")
    print("   • Content category analysis")
    print("   • Legal significance assessment")
    print("   • Feature detection (definitions, penalties, requirements)")
    print("   • Page-by-page importance mapping")
    print("   • AI confidence and quality metrics")
    
    print("\\n3. 🎯 Legal Analysis Features:")
    print("   • High importance section identification")
    print("   • Legal terminology detection")
    print("   • Cross-reference analysis")
    print("   • Language-aware processing")
    
    print("\\n💡 Usage for Malaysian Legal Acts:")
    print("   # Process single document")
    print("   results = process_pdf_with_ai_labeling('malaysian_acts/EN/act125.pdf')")
    print("   export_data = export_results_to_formats(results, 'parsed/EN')")
    print("   df = create_analysis_visualizations(results, 'parsed/EN')")
    
    return True

demonstrate_malaysian_legal_export()

print("\\n✅ Malaysian legal document export functions ready!")
print("?️  Optimized for legal document analysis and parsed folder structure")

## 🚀 Complete Processing Workflow

Now we'll put everything together into a complete workflow that processes your Malaysian legal PDFs with AI-powered content labeling.

In [ ]:
async def complete_malaysian_legal_analysis(
    pdf_path: str, 
    use_openai: bool = True,
    use_gemini: bool = False,
    output_dir: str = "/content/parsed",
    language: str = "auto"  # "EN", "BM", or "auto"
):
    """
    Complete workflow for analyzing Malaysian legal PDFs with AI labeling
    
    Args:
        pdf_path: Path to the PDF file (from malaysian_acts folder)
        use_openai: Whether to use OpenAI GPT-4 for labeling
        use_gemini: Whether to use Google Gemini for labeling
        output_dir: Directory to save results (parsed folder)
        language: Document language ("EN", "BM", or "auto" for detection)
    
    Returns:
        Dict containing complete analysis results
    """
    
    logger.info(f"🏛️  Starting Malaysian Legal PDF Analysis: {Path(pdf_path).name}")
    start_time = time.time()
    
    try:
        # Step 1: Detect language if auto
        if language == "auto":
            if '/EN/' in pdf_path.upper() or '/en/' in pdf_path.lower():
                language = "EN"
            elif '/BM/' in pdf_path.upper() or '/bm/' in pdf_path.lower():
                language = "BM"
            else:
                language = "other"
        
        print(f"🌐 Document language: {language}")
        
        # Step 2: Extract PDF content with unstructured
        print("📄 Step 1: Extracting PDF content...")
        elements = extract_pdf_content(pdf_path)
        if not elements:
            raise ValueError("No elements extracted from PDF")
        
        print(f"✅ Extracted {len(elements)} elements")
        
        # Step 3: Set up AI models
        print("🤖 Step 2: Setting up AI models...")
        
        clients = {}
        if use_openai and ai_system.openai_client:
            clients['openai'] = ai_system.openai_client
            print("✅ OpenAI GPT-4 ready")
        
        if use_gemini and ai_system.gemini_model:
            clients['gemini'] = ai_system.gemini_model
            print("✅ Google Gemini ready")
        
        if not clients:
            raise ValueError("No AI models available. Please configure API keys.")
        
        # Step 4: Process elements with AI labeling
        print("🏷️  Step 3: AI content labeling...")
        
        labeled_elements = []
        total_api_calls = 0
        
        for i, element in enumerate(elements):
            print(f"Processing element {i+1}/{len(elements)} (Page {element['page_number']})...", end="\\r")
            
            # Choose AI model (prefer OpenAI for legal documents)
            if 'openai' in clients:
                ai_labels = smart_label_content(ai_system, element['text'], element['element_type'], "openai")
                if ai_labels and 'error' not in ai_labels:
                    ai_labels['model_used'] = 'gpt-4'
                    total_api_calls += 1
            elif 'gemini' in clients:
                ai_labels = smart_label_content(ai_system, element['text'], element['element_type'], "gemini")
                if ai_labels and 'error' not in ai_labels:
                    ai_labels['model_used'] = 'gemini-pro'
                    total_api_calls += 1
            else:
                ai_labels = {}
            
            element['ai_labels'] = ai_labels
            labeled_elements.append(element)
            
            # Rate limiting
            if i % 10 == 0 and i > 0:
                time.sleep(1)  # Brief pause every 10 elements
        
        print(f"\\n✅ Completed AI labeling with {total_api_calls} API calls")
        
        # Step 5: Create analysis summary
        print("📊 Step 4: Generating analysis summary...")
        
        # Calculate statistics
        total_chars = sum(len(el['text']) for el in labeled_elements)
        content_categories = {}
        high_importance = 0
        category_distribution = {}
        
        for element in labeled_elements:
            ai_labels = element.get('ai_labels', {})
            category = ai_labels.get('content_category', 'unknown')
            significance = ai_labels.get('legal_significance', 'unknown')
            
            content_categories[category] = content_categories.get(category, 0) + 1
            
            if significance == 'high':
                high_importance += 1
        
        # Calculate percentages
        total_elements = len(labeled_elements)
        for category, count in content_categories.items():
            category_distribution[category] = round((count / total_elements) * 100, 1)
        
        # Step 6: Compile complete results
        processing_time = time.time() - start_time
        
        results = {
            'document_info': {
                'source_file': Path(pdf_path).name,
                'source_path': str(pdf_path),
                'language': language,
                'total_elements': total_elements,
                'total_characters': total_chars,
                'processing_time_seconds': round(processing_time, 2),
                'api_calls_made': total_api_calls,
                'models_used': list(clients.keys()),
                'processed_at': datetime.now().isoformat(),
                'extraction_method': 'unstructured_hi_res'
            },
            'labeled_elements': labeled_elements,
            'analysis_summary': {
                'content_categories': content_categories,
                'category_distribution': category_distribution,
                'high_importance_elements': high_importance,
                'average_confidence': round(
                    sum(el.get('ai_labels', {}).get('confidence_score', 0) for el in labeled_elements) / total_elements, 3
                ) if total_elements > 0 else 0
            }
        }
        
        # Step 7: Save to parsed folder with proper structure
        print("💾 Step 5: Saving to parsed folder...")
        
        # Create output directory structure
        parsed_path = Path(output_dir)
        lang_output_dir = parsed_path / language
        lang_output_dir.mkdir(parents=True, exist_ok=True)
        
        # Save complete JSON analysis
        output_filename = f"{Path(pdf_path).stem}_analysis.json"
        output_file = lang_output_dir / output_filename
        
        with open(output_file, 'w', encoding='utf-8') as f:
            json.dump(results, f, indent=2, ensure_ascii=False)
        
        print(f"✅ Analysis saved: {output_file}")
        
        # Step 8: Export additional formats
        print("📊 Step 6: Creating additional exports...")
        export_data = export_results_to_formats(results, str(lang_output_dir))
        
        # Step 9: Create visualizations
        print("📈 Step 7: Creating visualizations...")
        df = create_analysis_visualizations(results, str(lang_output_dir))
        
        logger.info(f"🎉 Analysis completed in {processing_time:.2f} seconds")
        logger.info(f"📊 Summary: {total_elements} elements, {high_importance} high importance")
        logger.info(f"📁 Saved to: {output_file}")
        
        return results
        
    except Exception as e:
        logger.error(f"❌ Analysis failed: {str(e)}")
        return None

# Updated demo function for Malaysian legal documents
def quick_malaysian_legal_demo():
    """Demonstrate the analysis workflow specifically for Malaysian legal documents"""
    
    print("?️  MALAYSIAN LEGAL PDF AI ANALYSIS SYSTEM")
    print("=" * 60)
    
    print("\\n📋 Specialized Features for Malaysian Legal Documents:")
    print("• 📄 PDF extraction optimized for legal formatting")
    print("• 🤖 AI models trained to understand legal terminology")
    print("• 🏛️  Malaysian legal document structure recognition")
    print("• 🌐 Language-aware processing (English/Bahasa Malaysia)")
    print("• 📊 Legal significance classification")
    print("• 💾 Structured output in JSON format for parsed folder")
    print("• ⚡ Batch processing for multiple acts")
    
    print("\\n📂 Folder Structure:")
    print("   📂 malaysian_acts/ (Input)")
    print("      📂 EN/ (English legal documents)")
    print("      📂 BM/ (Bahasa Malaysia legal documents)")
    print("   📂 parsed/ (Output)")
    print("      📂 EN/ (English analysis results)")
    print("      📂 BM/ (Bahasa Malaysia analysis results)")
    
    print("\\n🚀 Quick Start for Malaysian Legal Acts:")
    print("1. Ensure your malaysian_acts folder is uploaded to Colab")
    print("2. Configure API keys (OpenAI or Gemini)")
    print("3. Run batch processing or individual file analysis")
    print("4. Review results in the parsed folder")
    
    print("\\n💡 API Key Setup (Colab Secrets):")
    print("• In Colab: Go to 🔑 Secrets panel")
    print("• Add OPENAI_API_KEY and/or GEMINI_API_KEY")
    print("• Enable 'Notebook access' for each key")
    
    print("\\n⚠️  Processing Notes:")
    print("• Legal documents are processed with high-resolution extraction")
    print("• AI models provide specialized legal classification")
    print("• Results include legal significance ratings")
    print("• Processing time varies with document complexity")
    print("• API costs apply for AI analysis")
    
    print("\\n🎯 Legal Classification Categories:")
    print("• section_header, legal_definition, procedural_requirement")
    print("• penalty_clause, schedule, interpretation, preamble")
    print("• Legal significance: high, medium, low")
    print("• Subject matter: criminal_procedure, corporate_law, etc.")
    
    return True

quick_malaysian_legal_demo()

## 🧪 Testing Your Malaysian Legal PDFs

Ready to test the system! Upload your Malaysian legal PDF files and run the analysis.

In [ ]:
# 📁 STEP 1: Discover Malaysian Legal PDFs
# Scan the malaysian_acts folder for PDF files organized by language
import os
from pathlib import Path

def discover_malaysian_acts(base_dir: str = "/content/malaysian_acts"):
    """Discover and categorize Malaysian legal PDFs by language"""
    
    malaysian_acts_path = Path(base_dir)
    
    # Check if folder exists
    if not malaysian_acts_path.exists():
        print(f"❌ Folder not found: {base_dir}")
        print("📁 Please upload your malaysian_acts folder to Colab")
        print("   Expected structure:")
        print("   ? malaysian_acts/")
        print("      📂 EN/ (English documents)")
        print("      📂 BM/ (Bahasa Malaysia documents)")
        return {}
    
    discovered_files = {
        'EN': [],
        'BM': [],
        'other': []
    }
    
    # Scan for PDF files
    for root, dirs, files in os.walk(malaysian_acts_path):
        for file in files:
            if file.lower().endswith('.pdf'):
                file_path = Path(root) / file
                
                # Categorize by language folder
                if 'EN' in str(file_path).upper() or '/en/' in str(file_path).lower():
                    discovered_files['EN'].append(str(file_path))
                elif 'BM' in str(file_path).upper() or '/bm/' in str(file_path).lower():
                    discovered_files['BM'].append(str(file_path))
                else:
                    discovered_files['other'].append(str(file_path))
    
    # Display findings
    total_files = sum(len(files) for files in discovered_files.values())
    print(f"🔍 DISCOVERED MALAYSIAN LEGAL PDFS:")
    print(f"📂 Base directory: {base_dir}")
    print(f"📄 Total PDF files: {total_files}")
    print(f"   • English (EN): {len(discovered_files['EN'])} files")
    print(f"   • Bahasa Malaysia (BM): {len(discovered_files['BM'])} files")
    print(f"   • Other/Uncategorized: {len(discovered_files['other'])} files")
    
    # Show sample files
    if discovered_files['EN']:
        print(f"\\n? Sample EN files:")
        for file in discovered_files['EN'][:3]:
            print(f"   • {Path(file).name}")
    
    if discovered_files['BM']:
        print(f"\\n📄 Sample BM files:")
        for file in discovered_files['BM'][:3]:
            print(f"   • {Path(file).name}")
    
    return discovered_files

def setup_parsed_output_directory(base_dir: str = "/content/parsed"):
    """Setup the parsed output directory with language categorization"""
    
    parsed_path = Path(base_dir)
    parsed_path.mkdir(exist_ok=True)
    
    # Create language subdirectories
    (parsed_path / "EN").mkdir(exist_ok=True)
    (parsed_path / "BM").mkdir(exist_ok=True)
    (parsed_path / "other").mkdir(exist_ok=True)
    
    print(f"📁 PARSED OUTPUT DIRECTORY SETUP:")
    print(f"📂 Base directory: {base_dir}")
    print(f"   📂 EN/ (English analysis results)")
    print(f"   📂 BM/ (Bahasa Malaysia analysis results)")
    print(f"   📂 other/ (Other language results)")
    
    return parsed_path

# 🎯 STEP 2: Configure analysis for Malaysian legal documents
def configure_malaysian_legal_analysis():
    """Configure analysis specifically for Malaysian legal documents"""
    
    print("🏛️  MALAYSIAN LEGAL DOCUMENT ANALYSIS CONFIGURATION:")
    print("\\nDocument Types Expected:")
    print("• Acts of Parliament")
    print("• Regulations and Rules")
    print("• Legal Guidelines")
    print("• Constitutional Documents")
    
    # Check API key availability
    try:
        from google.colab import userdata
        openai_available = bool(userdata.get('OPENAI_API_KEY'))
        gemini_available = bool(userdata.get('GEMINI_API_KEY'))
    except:
        openai_available = bool(os.getenv('OPENAI_API_KEY'))
        gemini_available = bool(os.getenv('GEMINI_API_KEY'))
    
    print(f"\\n🔑 API Key Status:")
    print(f"   OpenAI: {'✅ Available' if openai_available else '❌ Not configured'}")
    print(f"   Gemini: {'✅ Available' if gemini_available else '❌ Not configured'}")
    
    if not openai_available and not gemini_available:
        print("\\n⚠️  No API keys configured!")
        print("Please add your API keys to Colab Secrets:")
        print("1. Click 🔑 in the left sidebar")
        print("2. Add OPENAI_API_KEY and/or GEMINI_API_KEY")
        print("3. Enable 'Notebook access'")
        return None, None
    
    # Recommend OpenAI for legal document analysis
    use_openai = openai_available
    use_gemini = not openai_available and gemini_available
    
    print(f"\\n🎯 Recommended Configuration:")
    print(f"   • OpenAI GPT-4: {'✅ Enabled' if use_openai else '❌ Disabled'}")
    print(f"   • Google Gemini: {'✅ Enabled' if use_gemini else '❌ Disabled'}")
    print(f"   • Optimized for Malaysian legal terminology")
    print(f"   • Language-aware processing (EN/BM)")
    
    return use_openai, use_gemini

# 🚀 STEP 3: Batch process Malaysian legal documents
async def process_malaysian_acts_folder(
    acts_dir: str = "/content/malaysian_acts",
    output_dir: str = "/content/parsed",
    language_filter: str = "all",  # "EN", "BM", or "all"
    max_files: int = None,
    max_pages_per_file: int = None
):
    """
    Process all Malaysian legal PDFs with language-aware analysis
    
    Args:
        acts_dir: Directory containing malaysian_acts
        output_dir: Directory to save parsed results
        language_filter: Process specific language ("EN", "BM", or "all")
        max_files: Maximum number of files to process per language
        max_pages_per_file: Maximum pages per file (None for all pages)
    """
    
    print("🏛️  MALAYSIAN LEGAL ACTS BATCH PROCESSING")
    print("=" * 60)
    
    # Step 1: Discover files
    discovered_files = discover_malaysian_acts(acts_dir)
    if not any(discovered_files.values()):
        print("❌ No PDF files found. Please check your folder structure.")
        return None
    
    # Step 2: Setup output directory
    parsed_path = setup_parsed_output_directory(output_dir)
    
    # Step 3: Configure AI models
    use_openai, use_gemini = configure_malaysian_legal_analysis()
    if use_openai is None:
        return None
    
    # Step 4: Filter files based on language preference
    files_to_process = []
    
    if language_filter == "all":
        for lang in ['EN', 'BM', 'other']:
            lang_files = discovered_files[lang]
            if max_files:
                lang_files = lang_files[:max_files]
            for file_path in lang_files:
                files_to_process.append((file_path, lang))
    elif language_filter in discovered_files:
        lang_files = discovered_files[language_filter]
        if max_files:
            lang_files = lang_files[:max_files]
        for file_path in lang_files:
            files_to_process.append((file_path, language_filter))
    
    if not files_to_process:
        print(f"❌ No files found for language filter: {language_filter}")
        return None
    
    print(f"\\n🎯 PROCESSING PLAN:")
    print(f"📄 Files to process: {len(files_to_process)}")
    print(f"🌐 Language filter: {language_filter}")
    print(f"📄 Max pages per file: {max_pages_per_file or 'All pages'}")
    
    # Step 5: Process files
    results = {}
    start_time = time.time()
    
    for i, (file_path, language) in enumerate(files_to_process):
        file_name = Path(file_path).name
        print(f"\\n📄 Processing {i+1}/{len(files_to_process)}: {file_name} ({language})")
        
        try:
            # Determine output file path
            output_file = parsed_path / language / f"{Path(file_path).stem}_analysis.json"
            
            # Skip if already processed (basic caching)
            if output_file.exists():
                print(f"   ⏭️  Already processed, loading from cache...")
                with open(output_file, 'r', encoding='utf-8') as f:
                    file_result = json.load(f)
                results[file_name] = file_result
                continue
            
            # Process with AI labeling
            file_result = process_pdf_with_ai_labeling(
                pdf_path=file_path,
                max_pages=max_pages_per_file,
                preferred_model="openai" if use_openai else "gemini",
                rate_limit_delay=1.0,
                batch_size=5
            )
            
            if file_result and 'document_info' in file_result:
                # Add language metadata
                file_result['document_info']['language'] = language
                file_result['document_info']['source_folder'] = str(Path(file_path).parent)
                
                # Save to parsed folder
                with open(output_file, 'w', encoding='utf-8') as f:
                    json.dump(file_result, f, indent=2, ensure_ascii=False)
                
                results[file_name] = file_result
                
                # Display quick summary
                info = file_result['document_info']
                summary = file_result['analysis_summary']
                print(f"   ✅ Processed: {info['total_elements']} elements in {info['processing_time_seconds']}s")
                print(f"   📊 High importance: {summary['high_importance_elements']} elements")
                
            else:
                print(f"   ❌ Processing failed for {file_name}")
                results[file_name] = {"error": "Processing failed"}
                
        except Exception as e:
            print(f"   ❌ Error processing {file_name}: {str(e)}")
            results[file_name] = {"error": str(e)}
    
    # Step 6: Generate batch summary
    total_time = time.time() - start_time
    successful = sum(1 for r in results.values() if "error" not in r)
    failed = len(results) - successful
    
    print(f"\\n🎉 BATCH PROCESSING COMPLETED!")
    print(f"✅ Successful: {successful}/{len(files_to_process)}")
    print(f"❌ Failed: {failed}")
    print(f"⏱️  Total time: {total_time:.1f} seconds")
    print(f"? Results saved in: {output_dir}")
    
    # Save batch summary
    batch_summary = {
        'batch_info': {
            'total_files': len(files_to_process),
            'successful': successful,
            'failed': failed,
            'total_processing_time': round(total_time, 2),
            'language_filter': language_filter,
            'processed_at': datetime.now().isoformat()
        },
        'file_results': results
    }
    
    summary_file = parsed_path / f"batch_summary_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
    with open(summary_file, 'w', encoding='utf-8') as f:
        json.dump(batch_summary, f, indent=2, ensure_ascii=False)
    
    print(f"📊 Batch summary saved: {summary_file}")
    
    return batch_summary

# 💡 Quick start examples for Malaysian legal documents
def show_malaysian_acts_examples():
    """Show practical usage examples for Malaysian legal documents"""
    
    print("💡 MALAYSIAN LEGAL ACTS - USAGE EXAMPLES:")
    print("\\n1. 📤 Process All Documents:")
    print("   results = await process_malaysian_acts_folder()")
    
    print("\\n2. ?🇧 English Documents Only:")
    print("   results = await process_malaysian_acts_folder(language_filter='EN')")
    
    print("\\n3. 🇲🇾 Bahasa Malaysia Documents Only:")
    print("   results = await process_malaysian_acts_folder(language_filter='BM')")
    
    print("\\n4. 🎛️  Limited Processing (for testing):")
    print("   results = await process_malaysian_acts_folder(")
    print("       language_filter='EN',")
    print("       max_files=5,")
    print("       max_pages_per_file=10")
    print("   )")
    
    print("\\n5. ? Custom Directories:")
    print("   results = await process_malaysian_acts_folder(")
    print("       acts_dir='/content/my_acts',")
    print("       output_dir='/content/my_parsed'")
    print("   )")
    
    print("\\n6. 🔍 Discover Files Only:")
    print("   files = discover_malaysian_acts('/content/malaysian_acts')")
    
    return True

show_malaysian_acts_examples()

print("\\n" + "="*60)
print("?️  READY TO PROCESS MALAYSIAN LEGAL ACTS!")
print("="*60)
print("\\n📂 Expected folder structure:")
print("   📂 malaysian_acts/")
print("      📂 EN/ (English legal documents)")
print("      📂 BM/ (Bahasa Malaysia legal documents)")
print("\\n📂 Output will be saved to:")
print("   📂 parsed/")
print("      📂 EN/ (English analysis results)")
print("      📂 BM/ (Bahasa Malaysia analysis results)")
print("\\n⚡ To start processing:")
print("   # Process all documents")
print("   results = await process_malaysian_acts_folder()")
print("\\n   # Process English only")
print("   results = await process_malaysian_acts_folder(language_filter='EN')")
print("\\n   # Test with limited files")
print("   results = await process_malaysian_acts_folder(max_files=3, max_pages_per_file=5)")